# Machine Learning: Home Work 1
<b>Leonardo Serilli</b>
- matricola 274426
- 25/11/2020

In [2]:
import pandas as pd
import numpy as np
import random
import sys 
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
#NOTE: you can change 'pd.read_csv()' argument and re-run just 
 # this and the last jupyer notebook blocks to try a new experiment on a different Data Set
DATA = pd.read_csv('Letters.csv')
features = [i for i in DATA]
alphabets = [DATA[i].unique().tolist() for i in DATA]

In [4]:
# return the encoded version of the value of x under the feature y 
def encode(x,y):
    charVect = []
    for i in range(len(alphabets[y])):
        if(alphabets[y][i]!=x):
            charVect.append(0)
        else:
            charVect.append(1)
    # notice here that without the 'index()' it would return the Characteristic Vector itself
    return charVect.index(1) 

def encodeDS(DS):
    encDS = []
    for index, row in DATA.iterrows():      
        encObservable = []
        for i in range(len(row)):
            encObservable.append(encode(row[i], features.index(row.index[i])))
        encDS.append(encObservable)
    return encDS

In [5]:
# substitute each missing value in the given test sets with the mean value of the feature where the NaN has been found
# NOTE: the way in wich models deal with missing values in prediction can be modified 
 # just changing the $fillna()$ argument
def nan_subs(nan_x_test):
    nan_x_test = pd.DataFrame(nan_x_test).fillna(nan_x_test.mean().astype(int))
    return nan_x_test

In [6]:
df = (pd.DataFrame(encodeDS(DATA)))
df_new = copy.deepcopy(df)

# NOTA: per ogni sample del dataset viene aggiunta al nuovo dataset, UNA SOLA COPIA,
 #con una feature casualmente inserita 
    # raddoppiando cosi la dimensione di df_new inserendovi una copia di ogni sample di df, con un ? introdotto su una (sola) feature casuale
for i, row in df.iterrows():
    new_sample = row
    new_sample[random.randint(0,16)] = np.nan
    df_new.loc[len(df)+i] = new_sample
    
# duplichiamo il dataset per avere un matching uno a uno dei due samples corrispondenti nel nuovo dataset
for i, row in df.iterrows():
    df.loc[len(df)+1] = row
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1
2,2,2,2,1,2,2,1,2,2,0,2,1,2,2,1,2,2
3,3,3,2,1,3,3,2,3,3,0,3,2,3,3,2,3,0
4,4,0,3,0,4,0,0,2,4,0,0,3,1,4,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39996,2,0,4,0,8,1,5,5,6,0,0,4,6,1,0,2,3
39997,14,3,9,5,2,4,9,4,4,2,7,8,12,1,5,2,3
39998,0,7,6,1,1,7,4,7,5,8,8,8,11,1,6,3,6
39999,5,0,10,3,6,0,0,5,2,0,2,4,0,4,5,4,0


In [15]:
# split in 80% TS e 20% testSet di entrambi i df
TS, testSet = train_test_split(df, test_size= 0.20, shuffle = False)
TS_new, testSet_new = train_test_split(df_new, test_size= 0.20, shuffle = False)

# costruzione di TS e testSet per i modelli, ogni join_TSets[i], avrà la feature i del df in OUT
 # e per IN ogni feature != i presa dal df_new (quindi con i ?) 
join_TSets = []
join_testSets = []

for i in range(len(list(df.columns))):
    
    tmp_TS_new = copy.deepcopy(TS_new)
    y = TS[i]
    x = tmp_TS_new.drop([i], axis=1)
    join_TSets.append({"x": x, "y": y})
    
    tmp_TestSet_new = copy.deepcopy(testSet_new)
    y = testSet[i]
    x = tmp_TestSet_new.drop([i], axis=1)
    join_testSets.append({"x": x, "y": y})

In [16]:
# training dei modelli, uno per ogni feature
models = [MLPClassifier(hidden_layer_sizes=(17,17,17), activation='relu', solver='adam', max_iter=1000) for i in range(len(list(df.columns)))]
for i in range(len(models)):
    models[i].fit(nan_subs(join_TSets[i]["x"]),join_TSets[i]["y"]) 

In [26]:
# DA FINIRE
acc_scores = []
for i in range(len(df.columns)):
    predictions = models[i].predict(nan_subs(join_testSets[i]["x"]))

    acc_scores.append(accuracy_score(predictions, join_testSets[i]["y"]))
    
for i in range(len(acc_scores)):
    print("\t\t",features[i], "\t=", "{:.2f}".format(acc_scores[i]))

		 lettr 	= 0.55
		 xbox 	= 0.59
		 ybox 	= 0.51
		 width 	= 0.54
		 high 	= 0.65
		 onpix 	= 0.49
		 xbar 	= 0.49
		 ybar 	= 0.48
		 x2bar 	= 0.40
		 y2bar 	= 0.40
		 xybar 	= 0.45
		 x2ybr 	= 0.46
		 xy2br 	= 0.47
		 xege 	= 0.53
		 xegvy 	= 0.59
		 yege 	= 0.47
		 yegvx 	= 0.50
